# Stable Video Diffusion

In [ ]:
# Download the necessary packages (globally)
!pip install torch accelerate transformers diffusers opencv-python

# Load Model
This load the pipeline and model artifacts into GPU memory

In [ ]:
import torch

from diffusers import StableVideoDiffusionPipeline
from diffusers.utils import load_image, export_to_video

pipe = StableVideoDiffusionPipeline.from_pretrained(
  "stabilityai/stable-video-diffusion-img2vid-xt", torch_dtype=torch.float16, variant="fp16"
)
pipe.enable_model_cpu_offload()

In [44]:
# Load the conditioning image
image = load_image("rohan-1.png")
image = image.resize((512, 512))
image

In [46]:
generator = torch.manual_seed(106)

## Create a Longer Video Iteratively

Create a sequence of video frames by iteratively generating new frames from an initial image. In each iteration, the last generated frame is used as the starting point for the next set of frames to ensure a smooth transition and continuity in the video. The frames are accumulated into a list, excluding the last frame of each iteration to avoid duplicates. This process allows for the creation of a longer, continuous video sequence from a single starting image.

In [ ]:
ITERATIONS = 3
all_frames = []
condition_image = image
for i in range(ITERATIONS):
    # generating frames based on a conditioning image
    frames = pipe(condition_image, decode_chunk_size=2, generator=generator, motion_bucket_id=90, noise_aug_strength=0.05).frames[0]
    # set last image of generated frames as the condition image for next iteration
    condition_image = frames[-1]
    # append frames (all except last one as it is used for conditioning in next iteration - to remove redundancy)
    all_frames += frames[:-1]

### Export Video

In [ ]:
export_to_video(all_frames, "rohan-it1-fp7.mp4", fps=7)